bank実装サンプル

matplotlibの設定

In [ ]:
!pip install japanize-matplotlib
import japanize_matplotlib

# matplotlibの図をセルに表示するための設定
%matplotlib inline

①

In [ ]:
# pandasのインポート
import pandas as pd

# CSVの読み込み
df1 = pd.read_csv('Bank_1.csv')
df2 = pd.read_csv('Bank_2.csv')

# 正しく読み込めているかを確認するため、それぞれのデータフレームの内容を表示する
print('===Bank_1.csv===')
print(df1.head())
print('行列数',df1.shape)
print('===Bank_2.csv===')
print(df2.head())
print('行列数',df2.shape)

===Bank_1.csv===
   id  age           job  marital  education default  amount housing
0   1   39   blue-collar  married  secondary      no    1756     yes
1   2   51  entrepreneur  married    primary      no    1443      no
2   3   36    management   single   tertiary      no     436      no
3   4   63       retired  married  secondary      no     474      no
4   5   31    management   single   tertiary      no     354      no
行列数 (27128, 8)
===Bank_2.csv===
   id loan   contact  day month    duration  campaign  previous  y
0   1   no  cellular    3   apr  370.055237         1         0  1
1   2   no  cellular   18   feb  233.998933        10         0  1
2   3   no  cellular   13   apr         NaN         1         2  0
3   4   no  cellular   25   jan  252.525808         1         0  0
4   5   no  cellular   30   apr         NaN         1         2  0
行列数 (27128, 9)


In [ ]:
# データフレームを'id'列をキーに列方向に連結する
df = df1.merge(right=df2, how='left', on='id')

# カラムが省略表示されないように設定を変更する
pd.set_option('display.max_columns', 100)

# 正しく連結できているかを確認するため、内容を表示する
print('===連結後のデータフレーム===')
print(df.head())
print('行列数',df.shape)

===連結後のデータフレーム===
   id  age           job  marital  education default  amount housing loan  \
0   1   39   blue-collar  married  secondary      no    1756     yes   no   
1   2   51  entrepreneur  married    primary      no    1443      no   no   
2   3   36    management   single   tertiary      no     436      no   no   
3   4   63       retired  married  secondary      no     474      no   no   
4   5   31    management   single   tertiary      no     354      no   no   

    contact  day month    duration  campaign  previous  y  
0  cellular    3   apr  370.055237         1         0  1  
1  cellular   18   feb  233.998933        10         0  1  
2  cellular   13   apr         NaN         1         2  0  
3  cellular   25   jan  252.525808         1         0  0  
4  cellular   30   apr         NaN         1         2  0  
行列数 (27128, 16)


②

In [ ]:
# 正解データの偏りの有無を確認するため、データの種類ごとの出現回数を表示する
df['y'].value_counts()

0    18445
1     8683
Name: y, dtype: int64

偏りがあることがわかった

③

In [ ]:
# 文字列データをダミー変数化する
# ダミー変数化の結果が変わらないよう、データ分割前にダミー変数化を実施する
# データフレームを丸ごと渡すことで、文字の列のみダミー変数化し、それ以外は元のままとしたデータフレームを取得できる
df_dummied = pd.get_dummies(df, drop_first=True)

# 正しくダミー変数化されていることを確認するため、内容を表示する
print(df_dummied.head())
print(df_dummied.shape)

   id  age  amount  day    duration  campaign  previous  y  job_blue-collar  \
0   1   39    1756    3  370.055237         1         0  1                1   
1   2   51    1443   18  233.998933        10         0  1                0   
2   3   36     436   13         NaN         1         2  0                0   
3   4   63     474   25  252.525808         1         0  0                0   
4   5   31     354   30         NaN         1         2  0                0   

   job_entrepreneur  job_housemaid  job_management  job_retired  \
0                 0              0               0            0   
1                 1              0               0            0   
2                 0              0               1            0   
3                 0              0               0            1   
4                 0              0               1            0   

   job_self-employed  job_services  job_student  job_technician  \
0                  0             0            0        

④

In [ ]:
# 各列に欠損値がどれだけ含まれているかを確認する
# 欠損値がある場合は穴埋めを考えるが、欠損値が多く分析で使えない列かもしれない
df_dummied.isnull().sum()

id                              0
age                             0
amount                          0
day                             0
duration                     7044
campaign                        0
previous                        0
y                               0
job_blue-collar                 0
job_entrepreneur                0
job_housemaid                   0
job_management                  0
job_retired                     0
job_self-employed               0
job_services                    0
job_student                     0
job_technician                  0
job_unemployed                  0
job_unknown                     0
marital_married                 0
marital_single                  0
education_secondary             0
education_tertiary              0
education_unknown               0
default_yes                     0
housing_yes                     0
loan_yes                        0
contact_sending _document       0
contact_telephone               0
month_aug     

In [ ]:
# 27128件中7044件もあるが、今回は穴埋めにより対処する
# 一旦は平均値で穴埋めすることとし、モデルの性能が良くなければ別の対処も考えることにする
df_fillna = df_dummied.fillna( df_dummied.mean() )

# 欠損値が補完されたことを確認する
df_fillna.isnull().any( axis=0 )

id                           False
age                          False
amount                       False
day                          False
duration                     False
campaign                     False
previous                     False
y                            False
job_blue-collar              False
job_entrepreneur             False
job_housemaid                False
job_management               False
job_retired                  False
job_self-employed            False
job_services                 False
job_student                  False
job_technician               False
job_unemployed               False
job_unknown                  False
marital_married              False
marital_single               False
education_secondary          False
education_tertiary           False
education_unknown            False
default_yes                  False
housing_yes                  False
loan_yes                     False
contact_sending _document    False
contact_telephone   

⑤

In [ ]:
# データを、特徴量、正解データに分ける
# 特徴量を一旦id、y以外のすべての列とする
X = df_fillna.drop(['id','y'], axis=1)

# 正解データは'y'
y = df_fillna['y']

print('===X===')
print(X.head())
print(X.shape)
print('===y===')
print(y.head())
print(y.shape)

===X===
   age  amount  day    duration  campaign  previous  job_blue-collar  \
0   39    1756    3  370.055237         1         0                1   
1   51    1443   18  233.998933        10         0                0   
2   36     436   13  318.222068         1         2                0   
3   63     474   25  252.525808         1         0                0   
4   31     354   30  318.222068         1         2                0   

   job_entrepreneur  job_housemaid  job_management  job_retired  \
0                 0              0               0            0   
1                 1              0               0            0   
2                 0              0               1            0   
3                 0              0               0            1   
4                 0              0               1            0   

   job_self-employed  job_services  job_student  job_technician  \
0                  0             0            0               0   
1                  0  

In [ ]:
# train_test_splitをインポートする
from sklearn.model_selection import train_test_split

# データを訓練データ＆検証データ、テストデータに分割する
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size = 0.2, random_state=0)

print('X_train_val:',X_train_val.shape )
print('X_test:',X_test.shape)

X_train_val: (21702, 38)
X_test: (5426, 38)


⑥

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

# ベースとして使用するモデルを深さ5の決定木(分類)　とする
base_model = DecisionTreeClassifier(random_state = 0, max_depth = 5, class_weight='balanced')

# アダブーストを使用してモデルを作成する
model = AdaBoostClassifier(n_estimators = 500, random_state = 0, base_estimator = base_model)
model.fit(X_train_val, y_train_val)

# テストデータでの正解率を確認する
print(model.score(X = X_test, y = y_test))

0.791927755252488


⑦

In [ ]:
# 学習で使用していない テストデータ の1行目で予測してみる
predict_data = X_test.iloc[:1, :]
predict_data

,age,amount,day,duration,campaign,previous,job_blue-collar,job_entrepreneur,job_housemaid,job_management,job_retired,job_self-employed,job_services,job_student,job_technician,job_unemployed,job_unknown,marital_married,marital_single,education_secondary,education_tertiary,education_unknown,default_yes,housing_yes,loan_yes,contact_sending _document,contact_telephone,month_aug,month_dec,month_feb,month_jan,month_jul,month_jun,month_mar,month_may,month_nov,month_oct,month_sep
22824,31,449,17,357.608637,1,11,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
model.predict(predict_data)

array([0])

特徴量重要度を確認しておく

In [ ]:
pd.Series(model.feature_importances_, index=X_train_val.columns).sort_values(ascending=False)

duration                     0.295606
amount                       0.218947
age                          0.115357
day                          0.096088
campaign                     0.051383
previous                     0.023735
contact_sending _document    0.013134
housing_yes                  0.012986
job_technician               0.012383
loan_yes                     0.011882
education_secondary          0.010999
marital_married              0.010153
job_services                 0.009956
month_may                    0.009755
month_jun                    0.009570
education_tertiary           0.009218
month_jul                    0.008766
marital_single               0.008527
job_blue-collar              0.008448
job_management               0.008218
month_nov                    0.007488
month_aug                    0.006857
contact_telephone            0.005205
job_self-employed            0.004814
job_entrepreneur             0.004203
education_unknown            0.003621
month_feb   